[Open In Colab](https://colab.research.google.com/github/joelbenjohn/pyCUFSM/blob/master/Validation.ipynb)

In [1]:
!rm -rf /content/pyCUFSM
!git clone https://github.com/ClearCalcs/pyCUFSM

import sys

sys.path.insert(1, "/content/pyCUFSM")
import numpy as np
from scipy import io
import pycufsm.fsm as fsm
import pycufsm.cfsm as cfsm
import pycufsm.helpers as helpers
import pycufsm.plotters as plotters
import pycufsm.preprocess as preprocess
from pycufsm.cutwp import prop2 as cutwp_prop2
import matplotlib.pyplot as plt

Cloning into 'pyCUFSM'...
remote: Enumerating objects: 696, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 696 (delta 93), reused 88 (delta 88), pack-reused 593
Receiving objects: 100% (696/696), 467.62 KiB | 312.00 KiB/s, done.
Resolving deltas: 100% (428/428), done.


ModuleNotFoundError: No module named 'pycufsm.cutwp_prop2'

In [ ]:
import glob


def getLocalFiles():
    filenames = glob.glob(sys.path + "?.mat")
    _files = {}
    if len(filenames) > 0:
        for file in filenames:
            open(file, "wb").write(_files[file])
    return _files


files = getLocalFiles()

In [ ]:
for k, v in files.items():
    file = io.loadmat(k)
    cufsm_input = helpers.load_cufsm_mat(file)
    sect_props = cutwp_prop2(cufsm_input["nodes"][:, 1:3], cufsm_input["elements"][:, 1:4])
    GBT_con = {
        "glob": cufsm_input["GBTcon"]["glob"][0][0][0],
        "dist": cufsm_input["GBTcon"]["dist"][0][0][0],
        "local": cufsm_input["GBTcon"]["local"][0][0][0],
        "other": cufsm_input["GBTcon"]["other"][0][0][0],
        "o_space": 1,
        "couple": 1,
        "orth": 2,
        "norm": 0,
    }
    signature, curve, basis = fsm.strip(
        props=cufsm_input["props"],
        nodes=cufsm_input["nodes"],
        elements=cufsm_input["elements"],
        lengths=cufsm_input["lengths"],
        springs=cufsm_input["springs"],
        constraints=cufsm_input["constraints"],
        GBT_con=GBT_con,
        B_C="S-S",
        m_all=np.ones((len(cufsm_input["lengths"]), 1)),
        n_eigs=10,
        sect_props=sect_props,
    )
    plt.semilogx(cufsm_input["lengths"], np.array(signature), "k", label="python")
    plt.semilogx(cufsm_input["lengths"], np.array(cufsm_input["curve"])[:, 1, 0], "ko", label="matlab")
    plt.xlim((1, np.max(cufsm_input["lengths"])))
    plt.ylim((0, np.min([np.max(signature), 3 * np.median(signature)])))
    plt.xlabel("half-wavelength (in)")
    plt.ylabel("Pcr/Py")
    plt.title(k[:-4])
    plt.show()